# To Do

* Greeks Calculator
* Payoff on Arbitrary Day

**Note:** Horizontal spreads don't work yet.

In [33]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import zipfile
import io
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# Run Plotly in offline mode
init_notebook_mode(connected=True)

In [34]:
###Automatic Data Cleaning starts
# Requires further refinement
from dateutil.parser import parse as date_parse

# Identify Floats
def is_float(inp):
    try:
        float(inp)
        return True
    except:
        return False

# Identify Ints
def is_int(inp):
    try:
        int(inp)
        return True
    except:
        return False

# Identify Dates
def is_date(inp):
    try:
        date_parse(inp)
        return True
    except:
        return False
    
#Clearning Function
def clean(inp):
    out = inp.strip()
    if(is_int(out)):
        out = int(out)
    elif(is_float(out)):
        out = float(out)
    return out
###Automatic Data Cleaning ends

# NSE Option Chain Scraper
def get_opt_chain(symbol, instrument, expiry):
    """symbol = nifty, banknifty, etc...;
    instrument = OPTIDX or OPTSTK;
    expiry = 19SEP2019 as a string"""
    # Convert input symbol string to uppercase
    symbol = symbol.upper()
    # Convert instrument type string to uppercase
    instrument = instrument.upper()
    # Convert expiry date string to uppercase
    expiry = expiry.upper()
    # Header of NSE website
    headers={'referer':"www.nseindia.com", 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    # Fetch source code of NSE Option Chain of given symbol, instrument and expiry
    req = requests.get("https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?segmentLink=17&instrument="+instrument+"&symbol="+symbol+"&date="+expiry, headers=headers)
    # Find table in the source code 
    table = bs(req.text).find('table',{'id':'octable'})
    # Columns names for table
    columns = ['call_oi', 'call_change_in_oi', 'call_traded_volume', 'call_iv', 'call_ltp', 'call_net_change', "call_bid_quantity", "call_bid_price", 'call_ask_price', 'call_ask_quantity', 'strike', 'put_bid_quantity', 'put_bid_price', 'put_ask_price', 'put_ask_quantity', 'put_net_change', 'put_ltp', 'put_iv', 'put_traded_volume', 'put_change_in_oi', 'put_oi']
    # Empty list to store data
    data = []
    # Iterate over rows in the table...
    for row in table.findAll('tr')[2:]:
        # Clean data in row and append to list
        data.append(list(map(lambda x : clean(x.text), row.findAll('td')[1:-1])))
        # Fill blank values with NaN
        while('-' in data[-1]):
            data[-1][data[-1].index("-")] = float('nan')
    # Return option chain as dataframe
    return pd.DataFrame(data, columns=columns).iloc[:-1].applymap(lambda x: float(str(x).replace(',','')))

In [35]:
# Class for Trade Legs
class Leg():
    
    
    
    # Initialize object parameters
    def __init__(self, expiry, qty, opt_type, strike):
        # Option Expiry
        self.expiry = expiry
        # Option Quantity
        self.qty = qty
        # Option Type
        self.opt_type = opt_type
        # option Strike
        self.strike = strike
        
        # Fetch Options Dataframe
        opt_df = get_opt_chain('banknifty', 'optidx', self.expiry)

        # Split dataframe into calls and puts
        calls = opt_df[opt_df.columns[:11]]
        puts = opt_df[opt_df.columns[10:]]
        
        ### Fetch LTP & IV from option chains ###
        
        # If call option...
        if self.opt_type == 'C':
            # Locate LTP of corresponding call strike
            self.entry_price = float(calls[calls['strike'] == self.strike]['call_ltp'].iloc[0])
            # Locate IV of corresponding call strike
            self.iv = float(calls[calls['strike'] == self.strike]['call_iv'].iloc[0])
        # If put option...
        elif self.opt_type == 'P':
            # Locate LTP of corresponding put strike
            self.entry_price = float(puts[puts['strike'] == self.strike]['put_ltp'].iloc[0])
            # Locate IV of corresponding put strike
            self.iv = float(puts[puts['strike'] == self.strike]['put_iv'].iloc[0])

        # Underlying Price Range
        underlying = np.array(strikes)
        
        ### Compute Payoff at Expiry ###
        
        # If Call option...
        if self.opt_type == 'C':
            # If long calls...
            if self.qty > 0:
                # payoff = max(underlying - strike - premium, -premium) ???
                self.expiry_payoff = np.where(underlying - self.strike - self.entry_price < - self.entry_price, - self.entry_price, underlying - self.strike - self.entry_price)
            # If short calls...
            elif self.qty < 0:
                # payoff = min(- underlying + strike + premium, premium) ???
                self.expiry_payoff = np.where(- underlying + self.strike + self.entry_price > self.entry_price, self.entry_price, - underlying + self.strike + self.entry_price)
        # If Put option...
        elif self.opt_type == 'P':
            # If long puts...
            if self.qty > 0:
                # payoff = max(strike - underlying - premium, -premium) ???
                self.expiry_payoff = np.where(self.strike - underlying - self.entry_price < - self.entry_price, - self.entry_price, self.strike - underlying - self.entry_price)
            # If short puts...
            elif self.qty < 0:
                # payoff = min(- strike + underlying + premium, premium) ???
                self.expiry_payoff = np.where(- self.strike + underlying + self.entry_price > self.entry_price, self.entry_price, -self.strike + underlying + self.entry_price)

    # Display Properties of Trade Leg
    def properties(self): print(self.expiry, self.qty, self.opt_type, self.strike, self.entry_price, self.iv)

In [36]:
# List of all Strikes in the chain
strikes = list(calls['strike'])

In [95]:
# Input number of legs in trade
no_of_legs = 8

In [103]:
# Initialize the legs of the trade
leg1 = Leg('19Sep2019', 3, 'C', 27200)
leg2 = Leg('19Sep2019', 3, 'P', 27200)
leg3 = Leg('19Sep2019', -2, 'C', 28500)
leg4 = Leg('19Sep2019', -2, 'P', 27100)
leg5 = Leg('19Sep2019', -2, 'C', 29200)
leg6 = Leg('19Sep2019', 2, 'P', 27100)
leg7 = Leg('19Sep2019', 1, 'C', 29500)
leg8 = Leg('19Sep2019', 1, 'P', 27000)

In [104]:
# Declare array to store leg payoffs
payoff = np.zeros(len(strikes))

# Iterate over legs and accumulate payoff
for x in range(no_of_legs):
    exec("payoff += leg%d.expiry_payoff*abs(leg%d.qty)" % (x+1, x+1))

In [105]:
# Plot interactive payoff graph
iplot([{"x": strikes, "y": payoff}])

In [106]:
# Maximum payoff at expiry
print(round(max(payoff), 2))

# Minimum payoff at expiry
print(round(min(payoff), 2))

4961.0
-2839.0
